<a href="https://colab.research.google.com/github/MarcoTheLoco/web3simplestorage/blob/main/delta_neutral_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Тут делаем некую симуляцию GBM для нашей цены по сути просто добавляя Random Walk

In [54]:
import pandas as pd
import numpy as np

days = 30
hours_per_day = 24
total_steps = days * hours_per_day
time_index = np.arange(total_steps)

np.random.seed(42)
price_changes = np.random.normal(0, 0.002, total_steps) # 0.2% волатильность в час
initial_price = 3000 # стартовая цена eth, можно изменять, для примера взято 3000
prices = initial_price * np.exp(np.cumsum(price_changes))
df = pd.DataFrame(data={'price': prices}, index=time_index)

Создаем датафрейм, указываем рендж цены, где наши деньги будут работать(ибо это ключевое отличие **Uniswap V3** от **Uniswap V2**, где деньги ходили от нуля до бесконечности).

Вот тут небольшой **Assumption** в виде того, что мы захардкодили и у нас ***Average Fee APR is estimated at 20% based on historical performance of ETH/USDC 0.05% pool***

Если брать реальную задачу, то нужны **API** ключи с *The Graph Uniswap V3 Subgraph*, потом писать запрос poolDayDatas на 0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640 (пул ETH/USDC) и получать *VolumeUSD* и *feesUSD* и это вставить вместо **fee_apr =**

In [55]:
initial_capital = 10000 # сколько денег планируем использовать
range_low = initial_price * 0.9 # -10%
range_high = initial_price * 1.1 # +10%
fee_apr = 0.20 # Предполагаемый APR от комиссий в год 20%

Хеджируем дельту позиции **Uniswap**, в нашем *Assumption* в начале LP позиция 50/50, значит экспозиция к ETH = 50% от капитала. Для торгового бота лучше высчитывать **Live Delta** каждую секунду для нормального подбора шорта **hedge_debt_eth**

Обращу внимание что в коде нет отдельной переменной impermanent loss, потому что il это часть стоимости изменения LP ибо
lp_value_no_fees = hodl_value * (1 + effective_il)
где **hodl_value** это сколько бы стоили деньги, если их просто держать в кошельке (без Uniswap),
а **effective_il** показывает, на сколько процентов уменьшилась стоимость LP из-за того, что цена ETH ушла от цены входа. Это отрицательное число. Умножая одно на другое, мы автоматически вычитаем IL.



In [56]:
hedge_debt_eth = (initial_capital / 2) / initial_price # Занимаем столько ETH
initial_debt_usd = hedge_debt_eth * initial_price

results = []
print_buffer = []

for t in range(total_steps):
    current_price = df.iloc[t]['price']

    # Расчет Uniswap Position Value (с учетом IL)
    price_ratio = current_price / initial_price

    # Немного упрощенная эвристика стоимости LP V3
    if range_low <= current_price <= range_high:
        # Классическая IL формула * Леверидж V3
        il_factor = (2 * np.sqrt(price_ratio) / (1 + price_ratio)) - 1
        v3_leverage = 3 # Примерный левередж для диапазона +/-10%
        effective_il = il_factor * v3_leverage
        # Базовая стоимость (холдим 50/50)
        hodl_value = (initial_capital / 2) * price_ratio + (initial_capital / 2)
        lp_value_no_fees = hodl_value * (1 + effective_il)
    else:
        # За пределами диапазона (упрощенно фиксируем убыток границы)
        if current_price > range_high:
             # Мы полностью в USDC (продали весь ETH рано)
             lp_value_no_fees = (initial_capital / 2) * (1 + (range_high/initial_price - 1)/2)
        else:
             # Мы полностью в ETH (купили падающий ETH)
             lp_value_no_fees = (initial_capital / 2) * (current_price/initial_price) * 2

    # Добавляем комиссии
    hourly_yield = fee_apr / 365 / 24
    accumulated_fees = initial_capital * hourly_yield * (t + 1)
    total_uniswap_value = lp_value_no_fees + accumulated_fees

    # Наш хедж (стоимость долга)
    current_debt_value = hedge_debt_eth * current_price
    # PnL от хеджа: (Сколько стоил долг в начале) - (Сколько стоит сейчас)
    # Если цена упала, долг стоит дешевле -> мы в плюсе
    hedge_pnl = initial_debt_usd - current_debt_value

    # Net PnL
    total_net_pnl = (total_uniswap_value - initial_capital) + hedge_pnl

    results.append({
        'Hour': t,
        'Price': current_price,
        'Uni Value': total_uniswap_value,
        'Debt Value': current_debt_value,
        'Net PnL': total_net_pnl
    })

print("\n" + "="*60)
print(f"--- BACKTEST REPORT ---")
print("="*60)
print(f"Strategy:        Uniswap V3 LP + Short ETH Hedge (Delta Neutral)")
print(f"Initial Capital: ${initial_capital}")
print(f"Price Range:     ${range_low:.2f} - ${range_high:.2f}")
print(f"Initial Debt:    {hedge_debt_eth:.4f} ETH (${initial_debt_usd:.2f})")
print("-" * 60)

res_df = pd.DataFrame(results)
pd.set_option('display.float_format', lambda x: '${:,.2f}'.format(x))

# Выбираем каждую 24-ю точку (раз в день) для краткости
display_df = res_df[res_df['Hour'] % 24 == 0].copy()

print(display_df.to_string(index=False))

print("-" * 60)
final_res = results[-1]
print(f"FINAL RESULTS (Day {days}):")
print(f"End Price: ${final_res['Price']:.2f}")
print(f"Net PnL:   ${final_res['Net PnL']:.2f}")
print("="*60)



--- BACKTEST REPORT ---
Strategy:        Uniswap V3 LP + Short ETH Hedge (Delta Neutral)
Initial Capital: $10000
Price Range:     $2700.00 - $3300.00
Initial Debt:    1.6667 ETH ($5000.00)
------------------------------------------------------------
 Hour     Price  Uni Value  Debt Value  Net PnL
    0 $3,002.98 $10,005.19   $5,004.97    $0.22
   24 $2,975.57  $9,964.75   $4,959.29    $5.46
   48 $2,943.48  $9,915.64   $4,905.79    $9.84
   72 $2,954.94  $9,940.72   $4,924.91   $15.81
   96 $2,938.15  $9,917.45   $4,896.92   $20.54
  120 $2,948.20  $9,940.16   $4,913.66   $26.50
  144 $2,932.42  $9,918.54   $4,887.36   $31.18
  168 $2,959.38  $9,970.19   $4,932.30   $37.89
  192 $2,966.28  $9,987.39   $4,943.80   $43.59
  216 $3,005.80 $10,059.19   $5,009.66   $49.53
  240 $2,991.80 $10,041.32   $4,986.33   $54.99
  264 $2,978.93 $10,025.19   $4,964.88   $60.32
  288 $2,982.56 $10,036.79   $4,970.94   $65.85
  312 $3,006.83 $10,082.83   $5,011.39   $71.44
  336 $3,031.87 $10,129.64   

*Сохранить в CSV:*

In [57]:
res_df.to_csv('delta_neutral_backtest.csv', index=False)
print(f"Full data saved to 'delta_neutral_backtest.csv'")

Full data saved to 'delta_neutral_backtest.csv'
